In [1]:
# load_ext autoreload
#autoreload 2
import numpy as np
import glob
from utils import *
import matplotlib.pyplot as plt
from matplotlib import colors
from sklearn import preprocessing

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from quantum_SVM import *   # QA SVM

In [2]:
outputpath='output/run_calibtrain'
maxalphas=20 # the 20 lowest-energy results returned by the quantum annealer are stored, but for the evaluation, we can consider less and compare

# Parameters 
Bs=[2,3] #[2,3,5,10]                      Base
Ks=[2] #[2,3]                             Number of qubits
xis=[0,1,5] #[0,1,5]                      Strength to consider the constraint
gammas=[-1] #[-1,0.125,0.25,0.5,1,2,4,8]  Kernel
Es=[0,1,2] #[0,1,2]                       Exponent
annealing_times=[1,10,100]
chain_strengths=[0.2,0.5,1,2,5]
embeddings=[0,1,2,3]


In [3]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))
from quantic.data import DatasetLoader

# network args
n_classes = 2
n_qubits = 2
n_features = None
if n_features is None:
    n_features = n_qubits
# train args
batch_size = 1
epochs = 10
LR = 0.001
n_samples_train = 100
n_samples_test = 50
# dataset args
shuffle = True
specific_classes_names = []
use_specific_classes = len(specific_classes_names)>=n_classes
dataset = "CIFAR10"
dataset_cfg = ""
print(specific_classes_names)

######## PREPARE DATASETS

# Train Dataset
# -------------

if dataset_cfg:
    # Load a dataset configuration file
    dataset = DatasetLoader.load(from_cfg=dataset_cfg,framework='torchvision')
else:
    # Or instantate dataset manually
    dataset = DatasetLoader.load(dataset_type=dataset,
                                   num_classes=n_classes,
                                   specific_classes=specific_classes_names,
                                   num_samples_class_train=n_samples_train,
                                   num_samples_class_test=n_samples_test,
                                   framework='torchvision'
                                   )
print(f'Dataset partitions: {dataset.get_partitions()}')

X_train = dataset['train']
X_test = dataset['test']

classes_str = ",".join(dataset.specific_classes_names)
classes2spec = {}
specific_classes = dataset.specific_classes
for idx, class_idx in enumerate(specific_classes):
    classes2spec[class_idx]=idx

classes_list = dataset.classes
n_samples = n_samples_test

[]
Files already downloaded and verified
Files already downloaded and verified
Dataset partitions: ['train', 'test']


In [4]:
X_train.data = X_train.data.reshape(X_train.data.shape[0],X_train.data.shape[1]*X_train.data.shape[2]*X_train.data.shape[3])

In [5]:
# generate qubos and couplers (embeddings)
# make sure you have signed up in dwavesys.com and registered your computer 'dwave config create' with profile (login) and api key
# https://cloud.dwavesys.com/leap/login/
for B in Bs:
    for K in Ks:
        for gamma in gammas:
            for xi in xis:
                for E in Es:
                    subpath=f"_B={B}_K={K}_xi={xi}_E={E}_gamma={float(gamma)}"
                    path = outputpath+subpath+"/"
                    gen_svm_qubos(X_train.data,X_train.targets,B,K,xi,gamma,E,path)
                    dwave_run_embedding(X_train.data,X_train.targets,path,annealing_times,chain_strengths,embeddings,solver={'qpu': True}) #solver='Advantage_system1.1' 

Extracting nodes and couplers
Saving 400 nodes and 19832 couplers for output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/
running output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/result_couplers=7500/ with 400 nodes and 7500 couplers for embedding [0, 1, 2, 3]
Running chain strength 0.2 and annealing time 1

 -- no embedding found, removing output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/result_couplers=7500/ and trying less couplers
running output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/result_couplers=5000/ with 400 nodes and 5000 couplers for embedding [0, 1, 2, 3]
Running chain strength 0.2 and annealing time 1

 -- no embedding found, removing output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/result_couplers=5000/ and trying less couplers
running output/run_calibtrain_B=2_K=2_xi=0_E=0_gamma=-1.0/result_couplers=2500/ with 400 nodes and 2500 couplers for embedding [0, 1, 2, 3]
Running chain strength 0.2 and annealing time 1

 -- no embedding found, removing output/run_calibtra

DisconnectedChainError: chain for 209 is not connected

In [ ]:
cmap = colors.ListedColormap(['black', 'red'])
plt.title('Prediction')
plt.rcParams["figure.figsize"] = (5,5)
Y_train_bin=np.where(Y_train==-1,0,Y_train)

for B in Bs:
    for K in Ks:
        for gamma in gammas:
            for xi in xis:
                for E in Es:
                    dirs=glob.glob(outputpath+f'_B={B}_K={K}_xi={xi}_E={E}_gamma={gamma}/result_couplers=*')
                    if not dirs:
                        dirs=glob.glob(outputpath+f'_B={B}_K={K}_xi={xi}_E={E}_gamma={float(gamma)}/result_couplers=*')
                    if len(dirs) == 0:
                        break
                    path=dirs[0]+'/'
                    f = open(path+f'collected_data_all_embeddings_maxalphas{maxalphas}.txt',"w") 
                    f.write("#rcs \tt_a \t trainacc\t trainF1score\t testacc\t testF1score\t average energy(train)\n") 
                    for emb in embeddings:
                        for c in chain_strengths:
                            for t in annealing_times:
                                alphas=np.load(path+f'embedding{emb}_rcs{c}_ta{t}_alphas.npy')
                                if not maxalphas == 0 or maxalphas > len(alphas):
                                    alphas = alphas[0:maxalphas]

                                scores_train=predict(X_train,X_train,Y_train,alphas,path)
                                Y_predict_train=np.sign(scores_train)
                                Y_predict_train=np.where(Y_predict_train==-1,0,Y_predict_train)
                                Y_predict_train=np.where(Y_predict_train==1,1,Y_predict_train)

                                scores=predict(X_test,X_train,Y_train,alphas,path)
                                Y_predict=np.sign(scores)
                                Y_predict=np.where(Y_predict==-1,0,Y_predict)   # From -1 to 0
                                Y_predict=np.where(Y_predict==1,1,Y_predict)    # From -1 to 1
        
                                trainacc = accuracy_score(Y_train_bin[:], Y_predict_train)
                                trainF1score = f1_score(Y_train_bin[:], Y_predict_train)
                                testacc = accuracy_score(Y_test[:], Y_predict)
                                testF1score = f1_score(Y_test[:], Y_predict)
                                alphas_avg = np.mean(alphas,axis=0)
                                av_energy = compute_energy(alphas_avg,X_train,Y_train,gamma,xi)
                
                                f.write(f'{c:1.2f}\t {t:4}\t {trainacc:8.4f}\t{trainF1score:8.4f}\t{testacc:8.4f}\t{testF1score:8.4f}\t{av_energy:8.4f}')
                                f.write("\n")

                                #  Visualize the prediction only for reasonable solutions
                                if testacc > 0.75 and testF1score > 0.7: 
                                    print(f'B = {B}, K = {K}, gamma = {gamma}, xi = {xi}, E = {E},\n  embedding {emb}, annealing time = {t}, rel. chain strength = {c}')
                                    #print('On train data:')
                                    #print ('Overal accuracy',trainacc)
                                    #print ('F1 score',trainF1score)
                                    print('Energy',av_energy)
                                    print('On test data')
                                    print ('Overal accuracy',testacc)
                                    print ('F1 score',testF1score)
                                    classification_map=np.reshape(Y_predict,(500,500))
                                    plt.imshow(classification_map, cmap=cmap)
                                    plt.clim(0, 1)
                                    plt.show()
                
                            f.write("\n")
                        f.write("\n")
                    f.close()
        